In [28]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install seaborn
import seaborn as sns
from datetime import *
import datetime as dt
from matplotlib.ticker import MaxNLocator
#!pip install regex
import regex
#!pip install emoji
import emoji
from seaborn import *
#!pip install heatmap
#from heatmap import heatmap
#!pip install wordcloud
#from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator
from nltk import *
#from plotly import express as px

### Python code to extract Date from chat file 

In [84]:
def startsWithDateAndTime(s):
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    pattern = '[0-9]{2}/[0-9]{2}/[0-9]{4}'
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [56]:

def FindAuthor(s):
    # ~~~~ Produce error ~~~~~ #
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [99]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    #if FindAuthor(message): 
    splitMessage = message.split(': ') 
    author = splitMessage[0] 
    message = ' '.join(splitMessage[1:])
    #else:
    #    author = None
    return date, time, author, message


In [100]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'GroupChat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    #fp.readline()
    #fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        #print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            #print(100)
            if len(messageBuffer) > 0:
                #print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            #print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

1146


In [101]:
df

,Date,Time,Author,Message
0,2021-11-08,7:20 pm,"Saad MS_34 created group ""2 dost or classmate 🤩😎""",
1,2021-11-08,7:20 pm,Saad MS_34 added you,
2,2021-11-08,7:21 pm,HR,Why you created this Group😄
3,2021-11-08,7:21 pm,Saad MS_34,😂
4,2021-11-08,7:22 pm,HR,Rockstar ko b add kr ly
...,...,...,...,...
1141,2021-08-14,2:04 pm,HR,Now Iqra on form 😄
1142,2021-08-14,2:12 pm,HR,Relx rahy Mam
1143,2021-08-14,2:12 pm,HR,🙌🏼
1144,2021-08-14,2:17 pm,Iqra MS_33,Her insan apny decision leny ka haq rkhta h so...


In [102]:
### Checking shape of dataset.
df.shape

(1146, 4)

In [103]:
### Checking basic information of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146 entries, 0 to 1145
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     1146 non-null   datetime64[ns]
 1   Time     1146 non-null   object        
 2   Author   1146 non-null   object        
 3   Message  1146 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 35.9+ KB


In [104]:
### Checking no. of null values in dataset
df.isnull().sum()

### Checking head part of dataset
#df.head(50)

### Checking tail part of dataset
#df.tail(50)

Date       0
Time       0
Author     0
Message    0
dtype: int64

In [112]:
### Droping Nan values from dataset
df = df.dropna()
df = df.reset_index(drop=True)

'''Don't drop, just take the rows where EPS is not NA'''

#df = df[~(df['Message'].str.len == 0)]
df = df.replace(r'^\s*$', np.nan , regex=True)
df.shape

(1146, 4)

In [113]:
### Checking no. of authors of group
df['Author'].nunique()

9

In [114]:
### Checking authors of group
df['Author'].unique()

array(['Saad MS_34 created group "2 dost or classmate 🤩😎"',
       'Saad MS_34 added you', 'HR', 'Saad MS_34', 'Iqra MS_33',
       'Saad MS_34 changed the subject from "2 dost or classmate 🤩😎" to "Hum 3 Dostt🤩🤩😎"',
       'Saad MS_34 added Ainee', 'Ainee',
       'You changed the subject from "Hum 3 Dostt🤩🤩😎" to "Idiots 🤪"'],
      dtype=object)

In [115]:
df.isna()
#df['Message']

,Date,Time,Author,Message
0,False,False,False,True
1,False,False,False,True
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
1141,False,False,False,False
1142,False,False,False,False
1143,False,False,False,False
1144,False,False,False,False


In [116]:
df = df[df['Message'].notna()]

In [118]:
df.shape

(1141, 4)

In [119]:
### Checking authors of group
df['Author'].unique()

array(['HR', 'Saad MS_34', 'Iqra MS_33', 'Ainee'], dtype=object)

### Adding one more column of "Day" for better analysis, here we use datetime library which help us to do this task easily.

In [122]:
weeks = {
0 : 'Monday',
1 : 'Tuesday',
2 : 'Wednesday',
3 : 'Thrusday',
4 : 'Friday',
5 : 'Saturday',
6 : 'Sunday'
}
df['Day'] = df['Date'].dt.weekday.map(weeks)
### Rearranging the columns for better understanding
df = df[['Date','Day','Time','Author','Message']]
### Changing the datatype of column "Day".
df['Day'] = df['Day'].astype('category')
### Looking newborn dataset.
#df.head()
#df.tail()

### Counting number of letters in each message
df['Letter\'s'] = df['Message'].apply(lambda s : len(s))
### Counting number of word's in each message
df['Word\'s'] = df['Message'].apply(lambda s : len(s.split(' ')))
### Function to count number of links in dataset, it will add extra column and store information in it.
URLPATTERN = r'(https?://S+)'
df['Url_Count'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.Url_Count)
### Function to count number of media in chat.
MEDIAPATTERN = r'<Media omitted>'
df['Media_Count'] = df.Message.apply(lambda x : re.findall(MEDIAPATTERN, x)).str.len()
media = np.sum(df.Media_Count)
### Looking updated dataset
df

,Date,Day,Time,Author,Message,Letter's,Word's,Url_Count,Media_Count
2,2021-11-08,Monday,7:21 pm,HR,Why you created this Group😄,27,5,0,0
3,2021-11-08,Monday,7:21 pm,Saad MS_34,😂,1,1,0,0
4,2021-11-08,Monday,7:22 pm,HR,Rockstar ko b add kr ly,23,6,0,0
5,2021-11-08,Monday,7:22 pm,HR,😏,1,1,0,0
6,2021-11-08,Monday,7:22 pm,Saad MS_34,😂😂😂,3,1,0,0
...,...,...,...,...,...,...,...,...,...
1141,2021-08-14,Saturday,2:04 pm,HR,Now Iqra on form 😄,18,5,0,0
1142,2021-08-14,Saturday,2:12 pm,HR,Relx rahy Mam,13,3,0,0
1143,2021-08-14,Saturday,2:12 pm,HR,🙌🏼,2,1,0,0
1144,2021-08-14,Saturday,2:17 pm,Iqra MS_33,Her insan apny decision leny ka haq rkhta h so...,66,15,0,0
